# Water Quality Project — Starter Notebook (No‑ML required)

**Covers:** data parsing → QA → trend → seasonality → exceedance → correlations → anomalies → segmentation → simple 2025 forecast preview

**Parameters of interest:** Fecal Coliform (MPN/100 mL), Turbidity (NTU)

**Scenarios:** ALL stations vs **NO_STP** (excludes STP inlet stations)


In [ ]:

import os, re, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
plt.rcParams.update({'figure.figsize': (8,4)})
BASE = Path('.'); DATA = BASE/'data'; OUT = BASE/'outputs'; OUT.mkdir(exist_ok=True)
excel_path = DATA/'DENZA 31 POINTS DATA 2020-2024 PHD.xlsx'
assert excel_path.exists(), f"Excel not found at {excel_path}"
print('Using data file:', excel_path)


In [ ]:

import pandas as pd
xls = pd.ExcelFile(excel_path)
YEARS = [s for s in xls.sheet_names if s.isdigit()]
def find_header_row(df, max_scan=60):
    for i in range(min(max_scan, len(df))):
        row_vals = df.iloc[i].astype(str).str.strip().str.lower().tolist()
        keys = ['sr. no', 'parameters', 'dissolved oxygen', 'turbidity', 'coliform', 'ph', 'bod']
        if any(any(k in v for k in keys) for v in row_vals):
            return i
    return None
header_rows = {s: find_header_row(pd.read_excel(excel_path, sheet_name=s, header=None)) for s in YEARS}
header_rows


In [ ]:

import datetime as pydt
def load_sheet(s, hdr):
    return pd.read_excel(excel_path, sheet_name=s, header=hdr)
tidy = {s: load_sheet(s, header_rows[s]) for s in YEARS}
def get_date_cols(cols):
    dcs = [c for c in cols if isinstance(c, (pd.Timestamp, pydt.datetime))]
    dcs = sorted(dcs, key=lambda x: (x.month if hasattr(x, 'month') else pd.Timestamp(x).month))
    return dcs
for s in YEARS:
    print(s, len(tidy[s]), 'rows; months:', [getattr(c,'month', None) for c in get_date_cols(tidy[s].columns)])


In [ ]:

import datetime as pydt, re, numpy as np, pandas as pd
def parse_sheet_to_long(year, df):
    date_cols = get_date_cols(df.columns)
    records = []; current_station = None
    for _, row in df.iterrows():
        sr = row.get('Sr. No'); param_label = row.get('Parameters')
        if pd.isna(param_label) and isinstance(sr, str) and len(sr.strip())>0:
            current_station = sr.strip(); continue
        if pd.isna(param_label) or current_station is None: continue
        p = str(param_label).strip().lower()
        if p.startswith('fecal coliform') or p.startswith('faecal coliform'): key = 'Fecal_Coliform'
        elif p.startswith('turbidity'): key = 'Turbidity'
        else: continue
        for dcol in date_cols:
            val = row[dcol]
            if isinstance(val, str):
                v = val.strip().lower()
                if v == 'bdl': num = 0.0
                else:
                    try: num = float(re.sub(r'[^0-9.\-eE]', '', v))
                    except: num = np.nan
            else:
                num = pd.to_numeric(val, errors='coerce')
            m = dcol.month if hasattr(dcol, 'month') else pd.Timestamp(dcol).month
            records.append({'year': int(year),'station': current_station,'parameter': key,'month': m,'date': pd.Timestamp(year=int(year), month=m, day=1),'value': num})
    return pd.DataFrame(records)

long_df = pd.concat([parse_sheet_to_long(s, tidy[s]) for s in YEARS], ignore_index=True)
long_df.loc[long_df['value'] < 0, 'value'] = np.nan
long_df['is_stp'] = long_df['station'].str.contains('STP', case=False, na=False)
long_df.head(), long_df.shape


In [ ]:

THRESH = {'Fecal_Coliform': 200.0, 'Turbidity': 5.0}
def annual_median(df):
    return (df.groupby(['parameter','year'])['value'].median().reset_index(name='annual_median'))
def yoy_change(ann):
    out = ann.copy(); out['yoy_change_pct'] = out.groupby('parameter')['annual_median'].pct_change()*100; return out
def exceedance_sample(df):
    return (df.groupby(['parameter','year']).apply(lambda g: (g['value'] > THRESH[g.name[0]]).mean()*100).reset_index(name='exceedance_rate_samples_pct'))


In [ ]:

def monthly_quarterly(df):
    monthly = (df.groupby(['parameter','year','month'], as_index=False)['value'].median().rename(columns={'value':'monthly_median'}))
    monthly['quarter'] = ((monthly['month']-1)//3)+1
    monthly['yoy_change'] = (monthly.sort_values(['parameter','month','year']).groupby(['parameter','month'])['monthly_median'].pct_change()*100)
    quarterly = (monthly.groupby(['parameter','year','quarter'], as_index=False)['monthly_median'].median().rename(columns={'monthly_median':'quarterly_median'}))
    quarterly['yoy_change'] = (quarterly.sort_values(['parameter','quarter','year']).groupby(['parameter','quarter'])['quarterly_median'].pct_change()*100)
    return monthly, quarterly

def plot_monthly_pattern(df, title):
    monthly, _ = monthly_quarterly(df)
    for param in monthly['parameter'].unique():
        pivot = monthly[monthly['parameter']==param].pivot(index='month', columns='year', values='monthly_median')
        ax = pivot.plot(kind='line', marker='o', title=f"{title} — {param}: Monthly medians by year")
        ax.set_xlabel("Month"); ax.set_ylabel("Median"); plt.tight_layout()
        fig_path = Path('outputs') / f"{title.replace(' ','_').lower()}_{param}_monthly_medians.png"
        plt.savefig(fig_path); plt.close()


In [ ]:

def monthly_series(df, param):
    s = (df[df['parameter']==param].groupby(['year','month'])['value'].median().rename('val'))
    ts = s.reset_index(); ts['date'] = pd.to_datetime(ts['year'].astype(str)+'-'+ts['month'].astype(str)+'-01')
    ts = ts.set_index('date')['val'].asfreq('MS'); return ts

def corr_and_leadlag(df):
    fc = monthly_series(df, 'Fecal_Coliform'); tb = monthly_series(df, 'Turbidity')
    corr = fc.corr(tb)
    shifts = range(-3,4); rows = []
    for k in shifts:
        if k<0: c = fc.shift(-k).corr(tb)  # tb leads
        else:   c = fc.corr(tb.shift(k))   # fc leads
        rows.append({'lag_months': k, 'corr': c})
    return corr, pd.DataFrame(rows)

def robust_z_log(series):
    x = np.log1p(series.dropna()); med = x.median(); mad = (x-med).abs().median()
    if mad == 0 or np.isnan(mad): return pd.Series(index=series.index, dtype=float)
    return ((x-med)/(1.4826*mad)).reindex(series.index)

def anomaly_flags(df, threshold=3.5):
    rows = []
    for param in df['parameter'].unique():
        g = df[df['parameter']==param]
        for st, sub in g.groupby('station'):
            m = (sub.groupby(['year','month'])['value'].median().rename('val').reset_index())
            m['date'] = pd.to_datetime(m['year'].astype(str)+'-'+m['month'].astype(str)+'-01')
            m = m.set_index('date').sort_index()
            z = robust_z_log(m['val']); m['robust_z_log'] = z; m['anomaly'] = (z.abs()>=3.5)
            m['parameter'] = param; m['station'] = st; rows.append(m.reset_index())
    return pd.concat(rows, ignore_index=True)

def segment_stations(df):
    res = []
    for param in df['parameter'].unique():
        g = df[df['parameter']==param]
        ex = (g.groupby('station').apply(lambda s: (s['value']>THRESH[param]).mean()*100).rename('exceed_pct'))
        med = g.groupby('station')['value'].median().rename('median')
        seg = pd.concat([ex, med], axis=1)
        def tier(row):
            if row['exceed_pct']>=60 or row['median']>=THRESH[param]*1.5: return 'High-Risk'
            if row['exceed_pct']>=30 or row['median']>=THRESH[param]:   return 'Moderate'
            return 'Lower'
        seg['tier']=seg.apply(tier, axis=1); seg['parameter']=param; res.append(seg.reset_index())
    return pd.concat(res, ignore_index=True)

def forecast_preview(df, param, year_next=2025):
    ts = (df[df['parameter']==param].groupby(['year','month'])['value'].median().rename('y').reset_index()).sort_values(['year','month'])
    ts['t']=np.arange(len(ts))+1
    for m in range(1,12): ts[f'm{m}']=(ts['month']==m).astype(int)
    import numpy as np
    X=ts[['t']+[f'm{m}' for m in range(1,12)]].values; y=ts['y'].values.reshape(-1,1)
    beta=np.linalg.pinv(X.T@X)@X.T@y; yhat=(X@beta).ravel(); resid=y.ravel()-yhat
    future=[{'year':year_next,'month':m,'t':int(ts['t'].iloc[-1])+i} | {f'm{k}':1 if m==k else 0 for k in range(1,12)} for i,m in enumerate(range(1,13), start=1)]
    F=pd.DataFrame(future); Xf=F[['t']+[f'm{k}' for k in range(1,12)]].values; yhat_f=(Xf@beta).ravel()
    s=resid.std(ddof=X.shape[1]); F['forecast']=yhat_f; F['lo80']=yhat_f-1.28*s; F['hi80']=yhat_f+1.28*s; F['lo95']=yhat_f-1.96*s; F['hi95']=yhat_f+1.96*s
    return F[['year','month','forecast','lo80','hi80','lo95','hi95']]


In [ ]:

SCENARIOS={'ALL': long_df.copy(), 'NO_STP': long_df[~long_df['is_stp']].copy()}
for name, df in SCENARIOS.items():
    print("\n=== Scenario:", name, "===")
    ann=annual_median(df); ann.to_csv(OUT/f"{name}_annual_median.csv", index=False)
    yoy=yoy_change(ann); yoy.to_csv(OUT/f"{name}_annual_median_yoy.csv", index=False)
    exc=exceedance_sample(df); exc.to_csv(OUT/f"{name}_exceedance_samples.csv", index=False)
    plot_monthly_pattern(df, name)
    an=anomaly_flags(df); an.to_csv(OUT/f"{name}_anomalies.csv", index=False)
    seg=segment_stations(df); seg.to_csv(OUT/f"{name}_station_segments.csv", index=False)
    for param in ['Fecal_Coliform','Turbidity']:
        fc=forecast_preview(df, param, year_next=2025)
        fc.to_csv(OUT/f"{name}_{param}_forecast_2025.csv", index=False)
print("\nSaved all outputs to outputs/")


In [ ]:

# Quick inline example plot: FC monthly medians by year (ALL)
monthly_all,_= (lambda d: (d.groupby(['parameter','year','month'], as_index=False)['value'].median().rename(columns={'value':'monthly_median'}), None))(SCENARIOS['ALL'])
param='Fecal_Coliform'
for y in sorted(monthly_all['year'].unique()):
    sub=monthly_all[(monthly_all['parameter']==param)&(monthly_all['year']==y)]
    plt.plot(sub['month'], sub['monthly_median'], marker='o', label=str(y))
plt.title("ALL — Fecal Coliform monthly medians by year"); plt.xlabel("Month"); plt.ylabel("Median"); plt.legend(); plt.tight_layout(); plt.show()
